# Credit Card Fraud Detection using Isolation Forest

This notebook has been prepared as part of the HSLU Fraud Detection Module in the MSc of Applied Information and Data Science.

## What is Isolation Forest?

Isolation Forest (Liu et al., 2008) is an anomaly detection method. There exist many more, but the algorithm is quite interesting in the way it works. Since anomalous data usually is sparse, anomaly detection algorithms oftentimes work by learning on clean data (learning the normal cases) and then is able to classify anomalous data due to it's *inability* to fit the anomalous data. Other candidates are for instance support vector machines, neural networks, etc.

Isolation Forest (IF) is based on decision trees. The general idea behind IF is that the algorithm recursively selects a feature of the data and then partitions the data by selecting a random value within said feature. By doing so, it tries to isolate each data point. It is intuitive that anomalous data points require fewer recursive steps than normal data points to be fully described.

As simple as the algorithm, it is striking yet how powerful it is. Despite it being introduced in 2008, it still has applications in todays ML world. There is a large amount of papers published in recent years<sup>1</sup>.

<sup>1</sup> Google Scholar shows 42k results since 2017 for Isolation Forest.

## Setup
First, let's load the data and have a look.

In [ ]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv(os.path.join('..', 'input', 'creditcardfraud', 'creditcard.csv'))
print(df.shape)
df.head()

We have a large dataset (284807 rows, 31 columns - incl. target). It contains a time offset column, an amount column and 28 encoded value columns. As we can see in the following plot (and output), the dataset is highly imbalanced. If we want to build a really powerful model, we would need to seriously consider the imbalance and apply appropriate techniques (such as SMOTE).

In [ ]:
print(df.Class.value_counts())
df.Class.value_counts().plot(kind='bar')

## Training

For the training, we will use a subset of the "non-fraudulent" class to train Isolation Forest. For the validation we will construct a set of data using both classes (equal split).

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

Let's first construct our datasets.

In an initial attempt, we will only train the isolation forest using a clean (normal) dataset.

In [ ]:
seed = 1337

def get_data(df, clean_train=True):
    """
        clean_train=True returns a train sample that only contains clean samples.
        Otherwise, it will return a subset of each class in train and test (10% outlier)
    """
    clean = df[df.Class == 0].copy().reset_index(drop=True)
    fraud = df[df.Class == 1].copy().reset_index(drop=True)
    print(f'Clean Samples: {len(clean)}, Fraud Samples: {len(fraud)}')

    if clean_train:
        train, test_clean = train_test_split(clean, test_size=len(fraud), random_state=seed)
        print(f'Train Samples: {len(train)}')

        test = pd.concat([test_clean, fraud]).reset_index(drop=True)

        print(f'Test Samples: {len(test)}')

        # shuffle the test data
        test.sample(frac=1, random_state=seed).reset_index(drop=True)
        
        train_X, train_y = train.loc[:, ~train.columns.isin(['Class'])], train.loc[:, train.columns.isin(['Class'])]
        test_X, test_y = test.loc[:, ~test.columns.isin(['Class'])], test.loc[:, test.columns.isin(['Class'])]
    else:
        clean_train, clean_test = train_test_split(clean, test_size=int(len(fraud)+(len(fraud)*0.9)), random_state=seed)
        fraud_train, fraud_test = train_test_split(fraud, test_size=int(len(fraud)*0.1), random_state=seed)
        
        train_samples = pd.concat([clean_train, fraud_train]).reset_index(drop=True)
        test_samples = pd.concat([clean_test, fraud_test]).reset_index(drop=True)
        
        # shuffle
        train_samples.sample(frac=1, random_state=seed).reset_index(drop=True)
        
        print(f'Train Samples: {len(train_samples)}')
        test_samples.sample(frac=1, random_state=seed).reset_index(drop=True)
        
        print(f'Test Samples: {len(test_samples)}')
        train_X, train_y = train_samples.loc[:, ~train_samples.columns.isin(['Class'])], train_samples.loc[:, train_samples.columns.isin(['Class'])]
        test_X, test_y = test_samples.loc[:, ~test_samples.columns.isin(['Class'])], test_samples.loc[:, test_samples.columns.isin(['Class'])]
    
    return train_X, train_y, test_X, test_y

train_X, train_y, test_X, test_y = get_data(df)

Now let's train our model.

In [ ]:
model = IsolationForest(random_state=seed)
model.fit(train_X, train_y)

Let's see how we do.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def predict(X):
    test_yhat = model.predict(X)
    # values are -1 and 1 (-1 for outliers and 1 for inliers), thus we will map it to 0 (inlier) and 1 (outlier) as this is our target variable
    test_yhat = np.array([1 if y == -1 else 0 for y in test_yhat])
    return test_yhat

test_yhat = predict(test_X)

In [ ]:
def get_confusion_matrix(test_y, test_yhat):
    cm = confusion_matrix(test_y, test_yhat)

    fig, ax = plt.subplots(1, 1,figsize=(10,10))

    tick_labels = ['Legitimate','Fraudulent']

    # calculate output
    total = np.sum(cm)
    tp = cm[0][0]
    fn = cm[0][1]
    fp = cm[1][0]
    tn = cm[1][1]
    tp_o = np.round((tp / total)*100,3)
    fn_o = np.round((fn / total)*100,3)
    fp_o = np.round((fp / total)*100,3)
    tn_o = np.round((tn / total)*100,3)
    tp_c = np.round(max((tp / (tp + fn)), 0.0)*100,3)
    fn_c = np.round(max((fn / (tp + fn)), 0.0)*100,3)
    fp_c = np.round(max((fp / (fp + tn)), 0.0)*100,3)
    tn_c = np.round(max((tn / (fp + tn)), 0.0)*100,3)
    labels = np.array([[
            f'TP\nCount: {tp}\nOverall: {tp_o}%\nTPR: {tp_c}%', 
            f'FN (Type 2)\nCount: {fn}\nOverall: {fn_o}%\nFNR: {fn_c}%'
        ],[
            f'FP (Type 1)\nCount: {fp}\nOverall: {fp_o}%\nFPR: {fp_c}%', 
            f'TN\nCount: {tn}\nOverall: {tn_o}%\nTNR: {tn_c}%'
        ]])
    sns.heatmap(cm, ax=ax, annot=labels, fmt='', cmap=plt.cm.RdYlBu)

    ax.set_title('Confusion Matrix', fontsize=14)

    ax.set_xticklabels(tick_labels, fontsize=14, rotation=0)
    ax.set_xlabel('Predicted', fontsize=20)
    ax.set_yticklabels(tick_labels, fontsize=14, rotation=0)
    ax.set_ylabel('Actual', fontsize=20)
    plt.show()

get_confusion_matrix(test_y, test_yhat)

In [ ]:
def get_classification_report(test_y, test_yhat):
    labels = ['Legitimate','Fraudulent']
    print(classification_report(test_y, test_yhat, target_names=labels))
get_classification_report(test_y, test_yhat)

As we can see from the classification report, we achieve an f1-score of 89% which is really good in an imbalanced data set.
Let's see how our model does if not only train on clean samples. Does it make a difference whether we train it with mixed clean + anomalous samples or not?

In [ ]:
train_X, train_y, test_X, test_y = get_data(df, clean_train=False)
model = IsolationForest(random_state=seed)
model.fit(train_X, train_y)
test_yhat = predict(test_X)

In [ ]:
get_confusion_matrix(test_y, test_yhat)

In [ ]:
get_classification_report(test_y, test_yhat)

It appears to be that this time around we perform a lot worse on the fraudulent cases (f1-score of 0.67). Overall, we are pretty good since we are now a lot more confident when predicting legitimate cases. However, since we want to be good in predicting fraudulent cases, this is not really acceptable.

# References

Liu, F. T., Ting, K. M., & Zhou, Z. H. (2008, December). Isolation forest. In 2008 eighth ieee international conference on data mining (pp. 413-422). IEEE.